In [2]:
import pandas as pd
import torch
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from utils import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
df_articles = load_newspaper()

In [ ]:
train_dataset, test_dataset = extract_train_test_dataset(df_articles)

In [7]:
# Set gloabal parameters and tokenizer
MAX_LEN = 512
batch_size = 16
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',do_lower_case=True)

In [8]:
test_dataloader = dataset_to_dataloader(test_dataset, tokenizer)
train_dataloader = dataset_to_dataloader(train_dataset, tokenizer)

In [11]:
# Load pretained model
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=7).to(device)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.weig

In [13]:
# extract parameters to optimize
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

opt = torch.optim.Adam(optimizer_grouped_parameters, lr=2e-5, eps=10e-8)

In [ ]:
model = train(model, train_dataloader, test_dataloader, opt, epochs=2)

In [16]:
torch.save(model.state_dict(), '../models/camembert_body.pt')